In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split

from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional, Lambda
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.models import Model
import keras.backend as K

W0528 21:45:07.712351  1112 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14
Using TensorFlow backend.


In [4]:
url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)


def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

Instructions for updating:
Colocations handled automatically by placer.


W0528 21:47:32.493178  1112 deprecation.py:323] From c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\tensorflow\python\ops\control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [15]:
from keras.layers import Dropout, RepeatVector, TimeDistributed

input_text1 = Input(shape=(1,), dtype=tf.string)
embedding1 = Lambda(UniversalEmbedding, output_shape=(512,))(input_text1)
dense1 = Dense(256, activation='relu')(embedding1)


input_text2 = Input(shape=(1,), dtype=tf.string)
embedding2 = Lambda(UniversalEmbedding, output_shape=(512,))(input_text2)
dense2 = Dense(256, activation='relu')(embedding2)

net1 = RepeatVector(3)(dense1)
net2 = RepeatVector(3)(dense2)

lstm_layer = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=.25))

x1 = lstm_layer(net1)
x2 = lstm_layer(net2)

merged = concatenate([x1, x2])
merged = BatchNormalization()(merged)
merged = Dropout(0.25)(merged)
merged = Dense(128, activation='relu')(merged)
merged = BatchNormalization()(merged)
merged = Dropout(0.25)(merged)
preds = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[input_text1, input_text2], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
model.summary()


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0529 01:16:15.442366  1112 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0529 01:16:16.885584  1112 saver.py:1483] Saver not created because there are no variables in the graph to restore


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
lambda_17 (Lambda)              (None, 512)          0           input_19[0][0]                   
__________________________________________________________________________________________________
lambda_18 (Lambda)              (None, 512)          0           input_20[0][0]                   
__________________________________________________________________________________________________
dense_9 (D